In [1]:
from pyspark import SparkContext, SparkConf
import os

In [2]:
DATA_DIR      = './Data'
OUTPUT_PATH   = './output'
FILENAME_LIST = os.listdir(DATA_DIR)
STOP_LIST     = ['they', 'she', 'he', 'it', 'the', 'as', 'is', 'and']

conf = SparkConf()
sc = SparkContext.getOrCreate(conf=conf)

21/12/01 02:54:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
# Read the words from files.
rdd_map = dict()
for filename in FILENAME_LIST:
    rdd = sc.textFile(os.path.join(DATA_DIR, filename))
    rdd = rdd.flatMap(lambda x : x.split()).map(lambda x : x.strip())
    rdd = rdd.filter(lambda x : x not in STOP_LIST)
    rdd = rdd.map(lambda x : (x, 1))
    rdd = rdd.reduceByKey(lambda x, y : x + y)
    rdd_map[filename] = rdd

In [4]:
# Union the rdds in `rdd_map`.
filename_list = list(rdd_map.keys())
rdd = sc.emptyRDD()
for filename in filename_list:
    rdd = rdd.union(
        rdd_map[filename].map(lambda x : (x[0], (x[1], filename)))
    )

In [5]:
rdd = rdd.map(lambda x : (x[0], [x[1]])) # Put the posting list into a (hash) set
rdd = rdd.reduceByKey(lambda x, y : x + y)

In [6]:
rdd.take(5)

[('Project', [(156, 'Hugo.tar.gz'), (156, 'Tolstoy.tar.gz')]),
 ('of',
  [(26964, 'Hugo.tar.gz'),
   (23449, 'Tolstoy.tar.gz'),
   (16535, 'shakespeare.tar.gz')]),
 ('Notre-Dame', [(45, 'Hugo.tar.gz')]),
 ('de',
  [(876, 'Hugo.tar.gz'),
   (157, 'Tolstoy.tar.gz'),
   (110, 'shakespeare.tar.gz')]),
 ('for',
  [(3587, 'Hugo.tar.gz'),
   (5768, 'Tolstoy.tar.gz'),
   (5974, 'shakespeare.tar.gz')])]

In [7]:
rdd.saveAsTextFile(OUTPUT_PATH)